# Estimating lower bounds for supervised machine learning

Martin Nilsson, RISE (mn@drnil.com)

The below Python program estimates how well a function can be learnt from a set of training data. It collects all observations in the neighborhood of a point, and computes the local standard deviation of the outputs corresponding to this set. The radius of the disc describes the regularity of the function, so that $r=0$ corresponds to complete overlearning (a very jagged function) and $r=\infty$ corresponds to a maximally smooth function, i.e., a constant equalling the mean of all observations.

Since training data is finite, there will always be a lack of data for small $r$. The extrapolation of the error to small $r$ is necessarily subjective.

For a detailed description of the mathematics, please refer to the corresponding Mathematica program.

We use the following libraries:

In [1]:
import numpy as np
import matplotlib.pyplot as mp

Below is the procedure. It expects five arguments:

* `inputData`, which is an array of input observations having one observation per row;
* `outputData`, which is a vector that has one element for every row of `inputData`;
* `t`, which is an index into `inputData` describing the local point of interest;
* `r`, a number describing the maximum radius; and
* `rstep`, which is a number giving the step size in the resulting diagram.

The output is a diagram estimating Bayes error as a function of `r`, surrounded by +/- 1 SD curves.

In [2]:
def lowerbounddiagram(inputData, outputData, t, r, rstep):
    # Whiten x
    m = np.mean(inputData)
    x = inputData.T - m
    a = x.dot(x.T)/x.shape[1]
    u, d, _ = np.linalg.svd(a)
    b = np.sqrt(np.linalg.pinv(np.diag(d))).dot(u.T)
    # whitened x in z
    z = b.dot(x)
    # Compute transformation of current location
    z0 = z[:,t]
    # Compute covariance of inputs with outputs
    cov = z.dot(outputData - np.mean(outputData))
    # Feature weighting from covariance
    weights = cov/np.sqrt(cov@cov)
    # Compute weighted distances from current point to other points
    dist = np.sqrt(np.sum((((z.T - z0) * weights).T)**2,axis=0))
    # Check n discs with radius from zero to r
    n = (int)(r/rstep)
    # Initialize result vector
    err = np.zeros((n), dtype=np.float64)
    pred = np.zeros((n), dtype=np.float64)
    regularity = np.zeros((n), dtype=np.float64)
    # Count of points in disc
    count = np.zeros((n), dtype=np.int32)
    regularity = rstep * np.arange(n)
    # For every disc,
    for i in range(n):
        disc = outputData[dist < regularity[i]]
        # Compute error
        if disc.size < 2: continue
        # only if there are at least two points within disc
        pred[i] = np.mean(disc)
        err[i] = np.std(disc)
        count[i] = disc.size
    return pred, regularity, err, count

For the following demo, we assume that observations are registered as a time series indexed by `t`. The delay from the last input observation to the output is `tdelay`. The data files are extracted from GEFCom'14 data (credits to Novin Shahroudi for this).

In [3]:
def demo(t, tdelay=0):
    global x
    global y
    try:
        x
    except :
        # Initialize x if not already initialized
        x = np.loadtxt("x2.csv", delimiter=",")
    try:
        y
    except:
        # Initialize x if not already initialized
        y = np.loadtxt("y2.csv", delimiter=",")
    if tdelay > 0:
        # Strip end of input data corresponding to delay
        x2 = x[:-tdelay,:]
    else:
        x2 = x
    # Strip beginning of output data corresponding to delay
    y2 = y[tdelay:]
    # Compute and plot error as a function of r
    pred, regularity, err, count = lowerbounddiagram(x2, y2, t, 2, 0.01)
    fig, ax = mp.subplots()
    delta = np.empty((count.size))
    delta[:] = np.NaN
    delta[count > 1] = 1/np.sqrt(count[count > 1]-1)
    ax.plot(regularity, err, lw=3)
    ax.plot(regularity, err * (1 + delta), 'r--')
    ax.plot(regularity, err * (1 - delta), 'r--')
    # Coefficient of variation
    #ax.plot(regularity, err/pred)
    ax.set(title='Estimate of best error as a function of\n'
                 'regularity with +/-1 SD',
           xlabel='Regularity',
           ylabel='Error')
    mp.show()

The following example shows the estimate given the wind speed forecast of four wind speed sensors 12 hours before and 12 hours after t (i.e., 96 pieces of data) at one wind power plant, predicting the output power:

In [4]:
demo(8000)

OSError: x2.csv not found.

Here, in a neighborhood of $t=8000$, it is reasonable to expect an error bound of approximately 0.05 given the present dataset and depending on how smooth a solution we expect.